In [3]:
!pip install openai-whisper
!pip install torch
!pip install pandas scikit-learn
!pip install language-tool-python

     ---------------------------------------- 0.0/800.5 kB ? eta -:--:--
     ------------- -------------------------- 262.1/800.5 kB ? eta -:--:--
     -------------------------------------- 800.5/800.5 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/894.9 kB ? eta -:--:--
   ---------------------------------------- 894.9/894.9 kB 5.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/204.1 MB ? eta -:--:--
   ---------------------------------------- 2.4/204.1 MB 11.2 MB/s eta 0:00:19
    --------------------------------------- 4.7/204.1 MB 10.9 MB/s eta 0:00:19
   - -------------------------------------- 7.3/204.1 MB 11.3 

In [36]:
import whisper
import language_tool_python
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore")

# Voice to Text using Whisper

In [14]:
def transcribe_audio(file_path):
    model = whisper.load_model("base")
    result = model.transcribe(file_path)
    return result["text"]


In [39]:
def transcribe_audio(file_path):
    # Check if file exists
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Audio file not found: {file_path}")
    
    try:
        model = whisper.load_model("base")
        result = model.transcribe(file_path)
        return result["text"]
    except Exception as e:
        print(f"Error during transcription: {e}")
        return ""


# Grammar Feature Extraction using LanguageTool

In [40]:
tool = language_tool_python.LanguageTool('en-US')

def extract_grammar_features(text):
    matches = tool.check(text)
    num_errors = len(matches)
    text_len = len(text.split())
    error_density = num_errors / text_len if text_len > 0 else 0
    return num_errors, error_density


# Prepare Dataset

In [41]:
# replace with real transcribed text & human scores
data = [
    {"text": "He go to school everyday", "score": 2},
    {"text": "He goes to school every day", "score": 5},
]

df = pd.DataFrame(data)
df[['num_errors', 'error_density']] = df['text'].apply(
    lambda x: pd.Series(extract_grammar_features(x))
)

# Train ML Model

In [42]:
X = df[['num_errors', 'error_density']]
y = df['score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor()
model.fit(X_train, y_train)

preds = model.predict(X_test)
print("MSE:", mean_squared_error(y_test, preds))

MSE: 9.0


# Full Prediction Pipeline

In [43]:
def grammar_score_from_audio(audio_path, trained_model):
    text = transcribe_audio(audio_path)
    num_errors, error_density = extract_grammar_features(text)
    score = trained_model.predict([[num_errors, error_density]])
    return text, score[0]


In [44]:
audio_file = "e:\Assingment\Grammer Voice Scoring\Dataset\Speaker26_000.wav"  
# Provide your audio file here"
text_output, predicted_score = grammar_score_from_audio(audio_file, model)

print(f"Transcribed Text: {text_output}")
print(f"Predicted Grammar Score: {predicted_score:.2f}")

Error during transcription: [WinError 2] The system cannot find the file specified
Transcribed Text: 
Predicted Grammar Score: 2.00
